In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

**1. Navigate to https://usapl.liftingdatabase.com/competitions. Using BeautifulSoup, extract information from each competition's website.** 

General notes from the competitions' default database:

- "All" for Type and State has the following HTML: https://usapl.liftingdatabase.com/competitions-default?t=&s=. 
- Changing the Type but keeping State as "All" changes the number after 't ='
    - T =  
        - International = 10 
        - IPF = 5
        - Local = 3
        - NAPF = 7
        - National = 2
        - Pro Meet = 9 
        - Pro Series = 8
        - Regional = 4
        - State = 6

**Code to retrieve results from a competition's url:**

In [2]:
def extract_category(td_tag):
    th_tag_before = td_tag.find_previous('th')  # Find the previous th tag
    th_tag_after = td_tag.find_next('th')  # Find the next th tag

    if th_tag_before and th_tag_after:
        # If the td tag is between two th tags, retrieve the previous th element
        return th_tag_before.get_text().strip()
    elif th_tag_before:
        # If the td tag is after a th tag, retrieve the previous th element
        return th_tag_before.get_text().strip()
    elif th_tag_after:
        # If the td tag is before a th tag, retrieve the next th element
        return th_tag_after.get_text().strip()
    else:
        return None

def retrieve_info(url):
    """
    This function creates a dataframe from the USAPL database.
    """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text)    

            results = []

            # Main html element where data is located 
            content = soup.find('div', id="content")

            if content:
                tables = content.findAll('table')

                if tables: 
                    meet_info = tables[0]
                    meet_results = tables[1]

                    # This is the table with competitor results
                    if meet_results:
                        tr_tags = meet_results.findAll('tr')

                        for tr_tag in tr_tags:
                            td_tags = tr_tag.findAll('td')

                            if len(td_tags) >= 2:
                                category = extract_category(td_tags[0])
                                weight_class = td_tags[0].get_text().strip().replace('-', '')
                                placement = td_tags[1].get_text().strip().replace('.', '')
                                name = td_tags[2].get_text().strip()
                                yob = td_tags[3].get_text().strip()
                                team = td_tags[4].get_text().strip()
                                state = td_tags[5].get_text().strip()
                                lot = td_tags[6].get_text().strip()
                                weight = td_tags[7].get_text().strip()
                                squat_1 = td_tags[8].get_text().strip()
                                squat_2 = td_tags[9].get_text().strip()
                                squat_3 = td_tags[10].get_text().strip()
                                bench_1 = td_tags[11].get_text().strip()
                                bench_2 = td_tags[12].get_text().strip()
                                bench_3 = td_tags[13].get_text().strip()
                                deadlift_1 = td_tags[14].get_text().strip()
                                deadlift_2 = td_tags[15].get_text().strip()
                                deadlift_3 = td_tags[16].get_text().strip()
                                total = td_tags[17].get_text().strip()
                                points = td_tags[18].get_text().strip()
                                drug_tested = td_tags[19].get_text().strip()

                                # Find the previous th tag for the event
                                event_tag = tr_tag.find_previous('th', 'competition_view_event')

                                if event_tag: 
                                    event = event_tag.get_text().strip()

                                else: 
                                    event = None

                                # Create a dictionary for each row
                                meet_results_data = {
                                    'Event': event,
                                    'Category': category,
                                    'Weight Class': weight_class,
                                    'Placement': placement,
                                    'Name': name,
                                    'Year of Birth': yob,
                                    'Team': team,
                                    'State': state,
                                    'Lot': lot,
                                    'Weight': weight,
                                    'Squat 1': squat_1,
                                    'Squat 2': squat_2,
                                    'Squat 3': squat_3,
                                    'Bench Press 1': bench_1,
                                    'Bench Press 2': bench_2,
                                    'Bench Press 3': bench_3,
                                    'Deadlift 1': deadlift_1,
                                    'Deadlift 2': deadlift_2,
                                    'Deadlift 3': deadlift_3,
                                    'Total': total,
                                    'Points': points,
                                    'Drug-Tested': drug_tested
                                }

                                # Append the dictionary to the results list
                                results.append(meet_results_data)

                        # Create a DataFrame from the results list
                        meet_results_df = pd.DataFrame(results)

                    # This is table with meet information
                    if meet_info:
                        tr_tags = meet_info.findAll('tr')

                        # Extracting date, state, and meet director information from tr_tags list
                        if len(tr_tags) >= 4:
                            date = tr_tags[0].find('td').get_text(strip=True)
                            sanction_num = tr_tags[1].find('td').get_text(strip=True)
                            state = tr_tags[2].find('td').get_text(strip=True)
                            meet_director = tr_tags[3].find('td').get_text(strip=True)

                        else: 
                            date = sanction_num = state = meet_director = None

                        # Repeat the values for each row in meet_results_df 
                        meet_info_df = pd.DataFrame({
                            'Meet Date': [date] * len(results),
                            'Sanction Number': [sanction_num] * len(results),
                            'Meet Location': [state] * len(results),
                            'Meet Director': [meet_director] * len(results)
                        }, index= meet_results_df.index)

                        # Concatenate the dataframes along the columns 
                        results_df = pd.concat([meet_info_df, meet_results_df], axis=1)

                    # Retrieve meet name from h3 heading
                    # Create a new column to add to results_df
                    meet_name = content.find('h3').get_text(strip=True)
                    meet_name_df = pd.DataFrame({
                        'Meet Name': [meet_name] * len(results)
                    })

                    # Concatenate the dataframes along the columns 
                    results_df = pd.concat([results_df, meet_name_df], axis = 1)                        

                    #Reorder columns in the DataFrame 
                    column_order = [
                        'Meet Date', 'Sanction Number', 'Meet Location', 'Meet Name', 'Meet Director', 
                        'Event', 'Category', 'Weight Class', 'Placement', 'Name', 
                        'Year of Birth', 'Team', 'State', 'Lot', 'Weight', 
                        'Squat 1', 'Squat 2', 'Squat 3', 'Bench Press 1', 
                        'Bench Press 2', 'Bench Press 3', 'Deadlift 1', 
                        'Deadlift 2', 'Deadlift 3', 'Total', 'Points', 'Drug-Tested'
                    ]
                    results_df = results_df[column_order]
                    
                    print('Success!')
                    return results_df

            else: 
                print(f'Error retrieving table element from {url}')
        else: 
            print(f'Error retrieving {url}')

    except Exception as e:
        print(f'Error retrieving data from {url}: {e}')

**Practice run with definition functions**

In [3]:
url = 'https://usapl.liftingdatabase.com/competitions-view?id=120652'
df = retrieve_info(url)
display(df)

Success!


,Meet Date,Sanction Number,Meet Location,Meet Name,Meet Director,Event,Category,Weight Class,Placement,Name,...,Squat 3,Bench Press 1,Bench Press 2,Bench Press 3,Deadlift 1,Deadlift 2,Deadlift 3,Total,Points,Drug-Tested
0,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Collegiate,75,1,Ellie Smith,...,-135,62.5,70,-75,125,140,152.5,347.5,343.64,
1,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Collegiate,75,2,Alexis Nelson,...,102.5,65,-70,-70,102.5,112.5,117.5,285,281.62,
2,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Collegiate,67.5,1,Kaileigh Estler,...,102.5,57.5,60,65,107.5,117.5,-127.5,285,300.26,
3,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Open,75,1,Maddie McKinley,...,-157.5,65,-70,-70,142.5,150,157.5,370,362.17,
4,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Open,100+,1,Rachel Parkhurst,...,80,40,45,50,112.5,117.5,120,250,204.61,
5,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Open,67.5,1,Marissa Weidner,...,-135,107.5,110,112.5,165,172.5,177.5,422.5,438.83,X
6,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Female - Raw Open,67.5,2,Erin Kaylor,...,125,70,75,80,102.5,120,127.5,332.5,359.35,
7,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Male - Raw Collegiate,90,1,Ethan Hensley,...,205,142.5,155,-160,245,260,272.5,632.5,414.69,
8,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Male - Raw Junior,90,1,BRANDON JONES,...,-297.5,185,197.5,-202.5,270,285,295,780,507.51,X
9,06/03/2023,TN-2023-02,Tennessee,Gateway to Champions,William Sandoval,Powerlifting,Male - Raw Junior,100,1,Joshua McCormick,...,-185,115,120,127.5,197.5,212.5,-227.5,505,325.27,


**2. Create DataFrame of all competition names and types in USAPL Database.** 

This DataFrame contains information of where and when the competition occurred along with the url that stores the results of the competition. 

In [4]:
list_of_urls = [
    'https://usapl.liftingdatabase.com/competitions-default?t=5&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=3&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=7&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=2&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=9&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=8&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=4&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=6&s='
]

competition_types = ['IPF', 'Local', 'NAPF', 'National', 
                     'Pro Meet', 'Pro Series', 'Regional', 'State']

all_competitions = []

# Go through list of URL and collect data
for url, competition_type in zip(list_of_urls, competition_types):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # List to collect information from each competition 
        competition_info = []
        
        # Retrieve general information from list of competitions
        tabledata = soup.find('table', 'tabledata')
        if tabledata:
            tr_tags = tabledata.findAll('tr')
            
            for tr_tag in tr_tags:
                td_tags = tr_tag.findAll('td')

                if td_tags:
                    date = td_tags[0].get_text().strip()
                    name = td_tags[1].get_text().strip()
                    sanction_num = td_tags[2].get_text().strip()
                    state = td_tags[3].get_text().strip()
                    
                    # Create a dictionary for each row
                    meet_type_data = {
                        'Date': date,
                        'Name': name,
                        'Meet Type': competition_type,
                        'Sanction Number': sanction_num,
                        'State': state
                    }
                    
                    # Append dictionary to results list
                    competition_info.append(meet_type_data)
                    
        else:
            print(f'Error retrieving URL: {url}')
            
        # Retrieve URL information
        anchor_tags = tabledata.findAll('a')
        url_list = ['https://usapl.liftingdatabase.com/' + anchor_tag.get('href') for anchor_tag in anchor_tags]

        # Update competition_info with Website info
        for url, meet_type_data in enumerate(competition_info):
            meet_type_data['Website'] = url_list[url]

        # Append competition_info to overall list 
        all_competitions.extend(competition_info)

# Create a DataFrame from the results list
competition_type_df = pd.DataFrame(all_competitions)
print('DataFrame created from list of USAPL competitions.')

# Display and save DataFrame as csv
competition_type_df

competition_type_df.to_csv('../data/USAPL_competitions.csv', index = False)
print('DataFrame saved as csv file in data folder.')


DataFrame created from list of USAPL competitions.
DataFrame saved as csv file in data folder.


**3. Using list of url's from the previous code, retrieve results from each competition using retrieve_info function.**

In [5]:
dfs = []

# Iterate through each complete URL
for url in url_list:
    df = retrieve_info(url)
    dfs.append(df)
    time.sleep(1)
    
# Concatenate all DataFrames into one
USAPL_powerlifting_df = pd.concat(dfs, ignore_index=True)

USAPL_powerlifting_df.to_csv('../data/usapl.csv', index = False)
print('DataFrame saved as csv file in data folder.')

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S

**4. Manually retrieve information from urls that did not work**

Errors: 
- Cannot open google spreadsheets (2)
    - https://usapl.liftingdatabase.com/https://docs.google.com/spreadsheets/d/1HnWXQGAxXTpjUVOP9FeEx92E6DcvbtIjXq0DwSqnO8I/edit?usp=sharing
    - https://usapl.liftingdatabase.com/https://docs.google.com/spreadsheets/d/1Al4TiaAWfMddLDETdiRyZIw3eElAlYTEvowd4ugzMcY/edit#gid=0
- 'NoneType' object has no attribute 'get_text' (5)
    - https://usapl.liftingdatabase.com/competitions-view?id=1033:
    - https://usapl.liftingdatabase.com/competitions-view?id=1038:
    - https://usapl.liftingdatabase.com/competitions-view?id=1249:
    - https://usapl.liftingdatabase.com/competitions-view?id=1632:
    - https://usapl.liftingdatabase.com/competitions-view?id=1684:
        - Resolved this issue by adding a None value to Event column in original retrieve_info function
- list index out of range (1) 
    - https://usapl.liftingdatabase.com/competitions-view?id=2011:
        - Used table[2] because table[1] is an 'Attachment' table
    

In [6]:
def extract_category(td_tag):
    th_tag_before = td_tag.find_previous('th')  # Find the previous th tag
    th_tag_after = td_tag.find_next('th')  # Find the next th tag

    if th_tag_before and th_tag_after:
        # If the td tag is between two th tags, retrieve the previous th element
        return th_tag_before.get_text().strip()
    elif th_tag_before:
        # If the td tag is after a th tag, retrieve the previous th element
        return th_tag_before.get_text().strip()
    elif th_tag_after:
        # If the td tag is before a th tag, retrieve the next th element
        return th_tag_after.get_text().strip()
    else:
        return None    
url = 'https://usapl.liftingdatabase.com/competitions-view?id=2011'
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text)    

    results = []

    # Main html element where data is located 
    content = soup.find('div', id="content")

    if content:
        tables = content.findAll('table')

        if tables: 
            meet_info = tables[0]
            meet_results = tables[2]

            # This is the table with competitor results
            if meet_results:
                tr_tags = meet_results.findAll('tr')

                for tr_tag in tr_tags:
                    td_tags = tr_tag.findAll('td')

                    if len(td_tags) >= 2:
                        category = extract_category(td_tags[0])
                        weight_class = td_tags[0].get_text().strip().replace('-', '')
                        placement = td_tags[1].get_text().strip().replace('.', '')
                        name = td_tags[2].get_text().strip()
                        yob = td_tags[3].get_text().strip()
                        team = td_tags[4].get_text().strip()
                        state = td_tags[5].get_text().strip()
                        lot = td_tags[6].get_text().strip()
                        weight = td_tags[7].get_text().strip()
                        squat_1 = td_tags[8].get_text().strip()
                        squat_2 = td_tags[9].get_text().strip()
                        squat_3 = td_tags[10].get_text().strip()
                        bench_1 = td_tags[11].get_text().strip()
                        bench_2 = td_tags[12].get_text().strip()
                        bench_3 = td_tags[13].get_text().strip()
                        deadlift_1 = td_tags[14].get_text().strip()
                        deadlift_2 = td_tags[15].get_text().strip()
                        deadlift_3 = td_tags[16].get_text().strip()
                        total = td_tags[17].get_text().strip()
                        points = td_tags[18].get_text().strip()
                        drug_tested = td_tags[19].get_text().strip()

                        # Find the previous th tag for the event
                        event_tag = tr_tag.find_previous('th', 'competition_view_event')
                        
                        if event_tag: 
                            event = event_tag.get_text().strip()
                            
                        else: 
                            event = None
                        
                        # Create a dictionary for each row
                        meet_results_data = {
                            'Event': event,
                            'Category': category,
                            'Weight Class': weight_class,
                            'Placement': placement,
                            'Name': name,
                            'Year of Birth': yob,
                            'Team': team,
                            'State': state,
                            'Lot': lot,
                            'Weight': weight,
                            'Squat 1': squat_1,
                            'Squat 2': squat_2,
                            'Squat 3': squat_3,
                            'Bench Press 1': bench_1,
                            'Bench Press 2': bench_2,
                            'Bench Press 3': bench_3,
                            'Deadlift 1': deadlift_1,
                            'Deadlift 2': deadlift_2,
                            'Deadlift 3': deadlift_3,
                            'Total': total,
                            'Points': points,
                            'Drug-Tested': drug_tested
                        }

                        # Append the dictionary to the results list
                        results.append(meet_results_data)
                
                # Create a DataFrame from the results list
                meet_results_df = pd.DataFrame(results)

            # This is table with meet information
            if meet_info:
                tr_tags = meet_info.findAll('tr')

                # Extracting date, state, and meet director information from tr_tags list
                if len(tr_tags) >= 4:
                    date = tr_tags[0].find('td').get_text(strip=True)
                    sanction_num = tr_tags[1].find('td').get_text(strip=True)
                    state = tr_tags[2].find('td').get_text(strip=True)
                    meet_director = tr_tags[3].find('td').get_text(strip=True)

                else: 
                    date = sanction_num = state = meet_director = None

                # Repeat the values for each row in meet_results_df 
                meet_info_df = pd.DataFrame({
                    'Meet Date': [date] * len(results),
                    'Sanction Number': [sanction_num] * len(results),
                    'Meet Location': [state] * len(results),
                    'Meet Director': [meet_director] * len(results)                
                })

                # Concatenate the dataframes along the columns 
                results_df = pd.concat([meet_info_df, meet_results_df], axis=1)

            # Retrieve meet name from h3 heading
            # Create a new column to add to results_df
            meet_name = content.find('h3').get_text(strip=True)
            meet_name_df = pd.DataFrame({
                'Meet Name': [meet_name] * len(results)
            })

            # Concatenate the dataframes along the columns 
            results_df = pd.concat([results_df, meet_name_df], axis = 1)                        

            #Reorder columns in the DataFrame 
            column_order = [
                'Meet Date', 'Meet Location', 'Meet Name', 'Meet Director', 
                'Event', 'Category', 'Weight Class', 'Placement', 'Name', 
                'Year of Birth', 'Team', 'State', 'Lot', 'Weight', 
                'Squat 1', 'Squat 2', 'Squat 3', 'Bench Press 1', 
                'Bench Press 2', 'Bench Press 3', 'Deadlift 1', 
                'Deadlift 2', 'Deadlift 3', 'Total', 'Points', 'Drug-Tested'
            ]
            PennState2018_df = results_df[column_order]

            print('Success!')
                
display(PennState2018_df)                

Success!


,Meet Date,Meet Location,Meet Name,Meet Director,Event,Category,Weight Class,Placement,Name,Year of Birth,...,Squat 3,Bench Press 1,Bench Press 2,Bench Press 3,Deadlift 1,Deadlift 2,Deadlift 3,Total,Points,Drug-Tested
0,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Bench press,Female - Raw Master 1,57,1,Karen Backenstose,1976,...,,60,65,-70,,,,65,76.69,
1,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Bench press,Male - Raw Master 2,105,1,Joseph Zwick,1957,...,,100,105,110,,,,110,66.48,
2,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Bench press,Male - Raw Master 3,74,1,Thomas Wilde,1948,...,,82.5,85,-90,,,,85,61.38,
3,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Bench press,Male - Raw Master 3,83,1,Sam Romeo,1955,...,,82.5,85,87.5,,,,87.5,62.82,
4,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Bench press,Male - Raw Master 3,105,1,Jim Mckenna,1948,...,,130,142.5,-147.5,,,,142.5,87.52,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Powerlifting,Male - Raw Teen 3,93,2,John Gamber,1998,...,195,137.5,142.5,-145,215,-225,,552.5,356.03,
136,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Powerlifting,Male - Raw Teen 3,105,1,Charlie Hechter,1999,...,245,127.5,137.5,142.5,250,265,277.5,665,403.59,X
137,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Powerlifting,Male - Raw Youth,40,1,Rowan Kratz,2006,...,62.5,27.5,32.5,-36.5,75,85,93,188,251.06,
138,04/07/2018,Pennsylvania,Pennsylvania States,Steve Mann,Powerlifting,Male - Raw Youth,53,1,Evan McCracken,2005,...,85,40,45,50,92.5,100,105,240,237.91,


**5. Combine USAPL_powerlifting_df and PennState2018_df**

In [12]:
USAPL_powerlifting_df = pd.concat([USAPL_powerlifting_df, PennState2018_df], axis = 0)

USAPL_powerlifting_df.to_csv('../data/usapl.csv', index = False)
print('DataFrame saved as csv file in data folder.')

DataFrame saved as csv file in data folder.
